In [1]:
print('hello')

hello


In [2]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-06-18 07:11:27--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-06-18 07:11:27--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-18 07:11:27 (28.6 MB/s) - ‘documents.json’ saved [658332

In [3]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [4]:
import json

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
len(documents)

948

In [6]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '5edd0a0e3438', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'zJLIX4rdQA-Lknmv7BLIqA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
# response = es.indices.create(index=index_name, body=index_settings)



In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/home/codespace/.local/share/virtualenvs/llm-zoomcamp-workshop-H1KjvqNQ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.47it/s]


In [11]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}



This Elasticsearch query is designed to search for documents based on the user's query within a specific course named "data-engineering-zoomcamp". Let's break down the components of the query:

```json
{
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
```

### Explanation:

1. **size**: 
   - Specifies the number of results to return. In this case, it's set to 5, meaning the query will return up to 5 documents that match the search criteria.

2. **query**:
   - The top-level key indicating that the following section defines the query to execute.

3. **bool query**:
   - A boolean query allows combining multiple query clauses to achieve specific search goals.
   - **must** clause:
     - Contains a `multi_match` query, which searches for documents that match the `user_question` in multiple fields (`question`, `text`, `section`). 
     - The `query` parameter is dynamically filled with `user_question` (presumably a user-provided search query).
     - `fields` specifies the fields to search in. The `^3` after `question` boosts its importance (weight) in the search results, indicating that matches in the `question` field are more significant.
     - `type` is set to `best_fields`, which finds documents where any field matches and uses the highest score for that match.

   - **filter** clause:
     - Contains a `term` query that filters documents based on exact matching of the `course` field.
     - Specifically, it filters documents where the `course` field exactly matches `"data-engineering-zoomcamp"`.

### Summary:
This Elasticsearch query will return up to 5 documents where:
- The documents must match the `user_question` in at least one of the specified fields (`question`, `text`, `section`).
- The `question` field is given higher relevance (`^3`) in determining matches.
- The documents must also have the `course` field set to `"data-engineering-zoomcamp"`.

This query structure is typical for searching specific fields with boosted relevance and applying filters to narrow down search results based on certain criteria.

In [12]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")


"""Retrieves top 5 matching documents.
Searches in the "question", "text", "section" fields, prioritizing "question".
Matches user query "How do I join the course after it has started?".
Shows results only for the "data-engineering-zoomcamp" course."""

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section

'Retrieves top 5 matching documents.\nSearches in the "question", "text", "section" fields, prioritizing "question".\nMatches user query "How do I join the course after it has started?".\nShows results only for the "data-engineering-zoomcamp" course.'

### Cleaning Up

In [13]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [14]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section

### Open AI 

In [15]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What's the formula for Energy?",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)


In physics, energy is often represented by the symbol U, E, or W. The formula for energy depends on the type of energy being described, but here are some common formulas:

1. **Kinetic Energy (KE)**: The energy of an object in motion is given by:

KE = (1/2)mv^2

where:
m = mass of the object
v = velocity of the object

2. **Potential Energy (PE)**: The energy an object has due to its position or configuration is given by:

PE = mgh

where:
m = mass of the object
g = acceleration due to gravity (approximately 9.8 m/s^2 on Earth)
h = height of the object above the ground

3. **Thermal Energy**: The energy of heat transfer is given by:

Q = mcΔT

where:
Q = heat transfer
m = mass of the substance being heated or cooled
ΔT = change in temperature

4. **Electromagnetic Energy**: The energy of light and other forms of electromagnetic radiation is given by:

E = hf

where:
E = energy of the photon
h = Planck's constant (approximately 6.626 x 10^-34 J s)
f = frequency of the light

5. **Mecha

### Building a prompt 

In [16]:
context_docs = retrieve_documents(user_question)

context = ""

for doc in context_docs:
    doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: G

In [17]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [18]:


client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

answer = chat_completion.choices[0].message.content
answer

'According to the provided context, the answer to the question "How do I join the course after it has started?" is:\n\n"Even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute."\n\nNote: The context does not provide a more specific instruction on how to join the course after it has started, only that it is possible to do so without registering and submitting homeworks.'

### CLEANING UP 

In [19]:
def build_context(documents):
    context = ""

    for doc in documents:
        doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context += doc_str
    
    context = context.strip()
    return context


def build_prompt(user_question, documents):
    context = build_context(documents)
    return f"""
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

def ask_openai(prompt, model="gpt-3.5-turbo"):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )
    
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    answer = response.choices[0].message.content
    return answer

def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_openai(prompt)
    return answer

In [20]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")



'The error message "repository name must be lowercase" suggests that you have an invalid reference format. To fix this issue, you can try to replace the "c:" with "//c:" and also try to replace the spaces in the path with no spaces.\n\nHere are a few options to replace the "-v" part:\n\n-v /c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n-v //c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n-v /c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n-v //c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n--volume //driveletter/path/ny_taxi_postgres_data/:/var/lib/postgresql/data\n\nAlso, try to add quotes around the directory path and make sure that there are no spaces or special characters in the path.\n\n-v "/c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data"\n-v "//c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data"\n-v “/c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data"\n-v "//c/some/path/ny_taxi_postgres_data:/var/